In [26]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime

Use this as a base setup for training a neural network on the data

# Dataset information - 

 ### Transaction Dataset information
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.
### Categorical Features:
- ProductCD
- card1 - card6
- addr1, addr2(billing region, country)
- P_emaildomain
- R_emaildomain
- M1 - M9



In [27]:
# This turns all the axes white in all the matplotlib plots. Comment this out if you dont want that
COLOR = 'white'
matplotlib.rcParams['text.color'] = COLOR
matplotlib.rcParams['axes.labelcolor'] = COLOR
matplotlib.rcParams['xtick.color'] = COLOR
matplotlib.rcParams['ytick.color'] = COLOR

### Train model

In [28]:
df_transaction = pd.read_csv('../datasets/ieee-fraud-detection/train_transaction.csv')

df_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Features that are used, isFraud is the target
features = ['isFraud', 'TransactionDT',
            'TransactionAmt','ProductCD', 'P_emaildomain','R_emaildomain']
features = features + [c for c in df_transaction.columns if c.startswith('C')]
features

['isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14']

In [30]:
df = df_transaction[features]
df.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14
0,0,86400,68.5,W,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0
1,0,86401,29.0,W,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
2,0,86469,59.0,W,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,0,86499,50.0,W,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0
4,0,86506,50.0,H,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0


In [31]:
target = 'isFraud'
# Categorical features
cat = ['TransactionDT','ProductCD', 'P_emaildomain','R_emaildomain']
# Numeric features
num = ['TransactionAmt'] + [c for c in df_transaction.columns if c.startswith('C')]

In [32]:
#Drop rows with missing features
df = df.dropna()
y = df[target].values

In [33]:
x_cat = df.filter(items = cat).values
x_num = df.filter(items = num).values

In [34]:
labelencoder_X = LabelEncoder()
# Label encode every categorical column
for i in range(len(cat)): 
    x_cat[:, i] = labelencoder_X.fit_transform(x_cat[:, i])

In [35]:
# Build input vector X, the training data
X = np.concatenate((x_cat, x_num), axis=1)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 0)

In [37]:
X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
X_val = np.asarray(X_val).astype('float32')
y_val = np.asarray(y_val).astype('float32').reshape((-1,1))

In [38]:
X_train.shape, y_train.shape

((80784, 19), (80784, 1))

### Train Model
Model:
Input -> Dense(Linear + activation) -> Dense -> Dense -> Output

In [39]:
# Initialize model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
2525/2525 [==============================] - 4s 1ms/step - loss: 12.2780 - accuracy: 0.8627
Epoch 2/10
2525/2525 [==============================] - 4s 1ms/step - loss: 0.2980 - accuracy: 0.9158
Epoch 3/10
2525/2525 [==============================] - 4s 2ms/step - loss: 0.3020 - accuracy: 0.9155
Epoch 4/10
2525/2525 [==============================] - 5s 2ms/step - loss: 0.2988 - accuracy: 0.9148
Epoch 5/10
2525/2525 [==============================] - 4s 2ms/step - loss: 0.3218 - accuracy: 0.9154
Epoch 6/10
2525/2525 [==============================] - 4s 1ms/step - loss: 0.2997 - accuracy: 0.9162
Epoch 7/10
2525/2525 [==============================] - 5s 2ms/step - loss: 0.2887 - accuracy: 0.9161
Epoch 8/10
2525/2525 [==============================] - 5s 2ms/step - loss: 0.3032 - accuracy: 0.9162
Epoch 9/10
2525/2525 [==============================] - 8s 3ms/step - loss: 0.2923 - accuracy: 0.9159
Epoch 10/10
2525/2525 [==============================] - 6s 2ms/step - loss: 0.29

In [40]:
val_loss, val_acc = model.evaluate(X_val, y_val)
print(val_loss, val_acc)

632/632 [==============================] - 1s 1ms/step - loss: 0.2853 - accuracy: 0.9173
0.2852809429168701 0.9173144698143005


Use this as starter code to getting a neural network up

Ideas going forward:
- Add more features to the model
- Use one-hot encoding instead of label-encoding
- Play with model architecture
- Figure out the meanings of hidden features
- Categorize the email features into common vs uncommon emails
- Hyperparamters( momentum? learning rate?)
- Data normalization